# Part One: Data Cleaning

University of Maryland - Info Challenge 2022

Team 32: Danny Rivas, Javan Reuto

Date: 03/05/22

---

First we begin, we'll first import libraries we will be using. 

In [1]:
import pandas as pd
import numpy as np
from numpy import nan
import re
import datetime as dt
import time
from functools import reduce
import requests
import plotly.express as px
import os

## Load the Data 

In [2]:
# Load files as dataframe
ga_full_df = pd.read_csv('ppp_applicants_ga_full.csv')
ga_removed_df = pd.read_excel('ppp-removed-ga.xlsx')

## Data Cleaning
To clean the data we created a function that first changes the case for our selected string columns. We changed data columns to datetime data. The function also changes the zip codes to 5 digits.

We applied this `cleaner()` function to both of the datasets that we imported.

In [3]:
# Creating function to clean dataframes - cleaner
def cleaner(dataframe):
    
    # Creating nested function to format string columns
    def uppercase_columns(df, column):
        df[column] = df[column].str.upper()
        return df
    
    # Invoking functions
    dataframe = uppercase_columns(dataframe, 'name')
    dataframe = uppercase_columns(dataframe, 'address')
    dataframe = uppercase_columns(dataframe, 'city')
    dataframe = uppercase_columns(dataframe, 'business_type')
    dataframe = uppercase_columns(dataframe, 'loan_status')
    dataframe = uppercase_columns(dataframe, 'lender')
    dataframe = uppercase_columns(dataframe, 'servicing_lender_name')
    dataframe = uppercase_columns(dataframe, 'servicing_lender_address')
    dataframe = uppercase_columns(dataframe, 'servicing_lender_city')
    dataframe = uppercase_columns(dataframe, 'business_age_description')
    dataframe = uppercase_columns(dataframe, 'project_city')
    dataframe = uppercase_columns(dataframe, 'project_county_name')
    dataframe = uppercase_columns(dataframe, 'originating_lender_city')
    
    # Creating nested function to format dates
    def format_date(df, column):
        df[column] = pd.to_datetime(df[column], format="%Y-%m-%d")
        return df
    
    # Invoking format_date function to date columns
    dataframe = format_date(dataframe,'date_approved')
    dataframe = format_date(dataframe,'loan_status_date')
    dataframe = format_date(dataframe,'forgiveness_date')

    def standard_zip(df,zip_col):
        df[zip_col] = df[zip_col].str[:5]
        return df

    dataframe = standard_zip(dataframe,'zip')
    dataframe = standard_zip(dataframe, 'servicing_lender_zip')
    dataframe = standard_zip(dataframe, 'project_zip')

    # Set loan number as index
    dataframe = dataframe.set_index('loan_number')
    
    # Return dataframe
    return dataframe

# Invoke function on dataframes
ga_full_df = cleaner(ga_full_df)
ga_removed_df = cleaner(ga_removed_df)



Because all the columns in both datasets are the same, we can combine them for further cleaning.

In [4]:
# Adding dummy variable for removed dataset
ga_full_df["Removed"] = 0
ga_removed_df["Removed"] = 1

#Combining both datasets in ga_df
ga_df = pd.concat([ga_full_df, ga_removed_df]).reset_index()

del ga_full_df
del ga_removed_df

ga_df

,loan_number,name,amount,state,address,city,zip,naics_code,business_type,jobs_retained,...,project_state,project_zip,originating_lender_city,originating_lender_state,loan_status_date,originating_lender_location_id,lmi_indicator,forgiveness_amount,forgiveness_date,Removed
0,2519198508,LUMMUS CORPORATION,2000000.0,GA,225 BOURNE BLVD,SAVANNAH,31408,333249.0,CORPORATION,294,...,GA,31408,CHICAGO,IL,NaT,116998,N,NaN,NaT,0
1,9918008300,COLIANT SOLUTIONS INC.,1294555.0,GA,2703 BRICKTON NORTH DR,BUFORD,30518,541519.0,CORPORATION,63,...,GA,30518,COLUMBUS,GA,NaT,19248,Y,NaN,NaT,0
2,9662228509,YOHE PLUMBING INC,729509.0,GA,1120 FRANKE COURT N/A,AUGUSTA,30909,238220.0,CORPORATION,105,...,GA,30909,PHOENIXVILLE,PA,NaT,121536,N,NaN,NaT,0
3,1661788406,LEWIS COLOR LITHOGRAPHERS INC,571193.4,GA,30 JOE KENNEDY BLVD,STATESBORO,30458,323111.0,CORPORATION,49,...,GA,30458,COLUMBUS,GA,NaT,19248,N,NaN,NaT,0
4,1217957101,"ALMA PAK INTERNATIONAL, LLC",472700.0,GA,230 PINEVIEW RD,ALMA,31510,445230.0,PARTNERSHIP,37,...,GA,31510,ALMA,GA,2021-01-08,17811,N,475877.59,2020-12-15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579659,4662698703,DONNELL WILSON,1950.0,GA,1000 PACES LN,WOODSTOCK,30189,621511.0,SOLE PROPRIETORSHIP,1,...,GA,30189,LAGUNA HILLS,CA,2021-04-01,509316,N,NaN,NaT,1
579660,5057699009,MAMADOU DIOULDE DIALLO,1849.0,GA,7000 ROSWELL RD APT 252,ATLANTA,30328,485310.0,INDEPENDENT CONTRACTORS,1,...,GA,30328,BEDFORD,TX,2021-05-21,529472,N,NaN,NaT,1
579661,8820408702,ANTHONY HARRIS,1245.0,GA,2849 BLOUNT ST,ATLANTA,30344,485310.0,INDEPENDENT CONTRACTORS,1,...,GA,30344,BEDFORD,TX,2021-04-08,529472,Y,NaN,NaT,1
579662,8028788903,TAAHIRA ROBINSON,1075.0,GA,8541 GLENDEVON CT,RIVERDALE,30274,561110.0,LIMITED LIABILITY COMPANY(LLC),1,...,GA,30274,MIAMI,FL,2021-05-11,529192,Y,NaN,NaT,1


### Location Cleaning

We noticed that some cities had names that seemed like street names. We decided to create the function `standard_city()` that changes city names based on zip codes using a dictionary. We also included a timer to keep track how long the function takes to run for each city cleaning

In [5]:
# Create function to clean city names and zip codes
def standard_city(df, zip_col, city_col):
    start = time.process_time() #Start timer
    city_dict = dict(zip(df[zip_col], df[city_col])) # Create dictionary
    df[city_col] = df[zip_col] 
    df[city_col] = df[city_col].replace(city_dict) # Replace city values with name from dictionary
    print(f"{city_col} standardized in {(time.process_time() - start):.2f} seconds.") # End timer
    return df

# Invoking function on all our zip codes and cities
ga_df = standard_city(ga_df,'zip', 'city')
ga_df = standard_city(ga_df, 'servicing_lender_zip','servicing_lender_city')
ga_df = standard_city(ga_df, 'project_zip', 'project_city')

ga_df

city standardized in 13.36 seconds.
servicing_lender_city standardized in 12.98 seconds.
project_city standardized in 13.97 seconds.


,loan_number,name,amount,state,address,city,zip,naics_code,business_type,jobs_retained,...,project_state,project_zip,originating_lender_city,originating_lender_state,loan_status_date,originating_lender_location_id,lmi_indicator,forgiveness_amount,forgiveness_date,Removed
0,2519198508,LUMMUS CORPORATION,2000000.0,GA,225 BOURNE BLVD,SAVANNAH,31408,333249.0,CORPORATION,294,...,GA,31408,CHICAGO,IL,NaT,116998,N,NaN,NaT,0
1,9918008300,COLIANT SOLUTIONS INC.,1294555.0,GA,2703 BRICKTON NORTH DR,BUFORD,30518,541519.0,CORPORATION,63,...,GA,30518,COLUMBUS,GA,NaT,19248,Y,NaN,NaT,0
2,9662228509,YOHE PLUMBING INC,729509.0,GA,1120 FRANKE COURT N/A,AUGUSTA,30909,238220.0,CORPORATION,105,...,GA,30909,PHOENIXVILLE,PA,NaT,121536,N,NaN,NaT,0
3,1661788406,LEWIS COLOR LITHOGRAPHERS INC,571193.4,GA,30 JOE KENNEDY BLVD,STATESBORO,30458,323111.0,CORPORATION,49,...,GA,30458,COLUMBUS,GA,NaT,19248,N,NaN,NaT,0
4,1217957101,"ALMA PAK INTERNATIONAL, LLC",472700.0,GA,230 PINEVIEW RD,ALMA,31510,445230.0,PARTNERSHIP,37,...,GA,31510,ALMA,GA,2021-01-08,17811,N,475877.59,2020-12-15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579659,4662698703,DONNELL WILSON,1950.0,GA,1000 PACES LN,WOODSTOCK,30189,621511.0,SOLE PROPRIETORSHIP,1,...,GA,30189,LAGUNA HILLS,CA,2021-04-01,509316,N,NaN,NaT,1
579660,5057699009,MAMADOU DIOULDE DIALLO,1849.0,GA,7000 ROSWELL RD APT 252,ATLANTA,30328,485310.0,INDEPENDENT CONTRACTORS,1,...,GA,30328,BEDFORD,TX,2021-05-21,529472,N,NaN,NaT,1
579661,8820408702,ANTHONY HARRIS,1245.0,GA,2849 BLOUNT ST,ATLANTA,30344,485310.0,INDEPENDENT CONTRACTORS,1,...,GA,30344,BEDFORD,TX,2021-04-08,529472,Y,NaN,NaT,1
579662,8028788903,TAAHIRA ROBINSON,1075.0,GA,8541 GLENDEVON CT,RIVERDALE,30274,561110.0,LIMITED LIABILITY COMPANY(LLC),1,...,GA,30274,MIAMI,FL,2021-05-11,529192,Y,NaN,NaT,1


While zip codes were used to clean the city names, we now do the opposite to create a dictionary based on city names in order to clean them. We created a dictionary based on a **all** zip codes available in our dataset. After cleaning zip codes, we changed the values to strings.

In [6]:
#Creating function to build zip code dictionary
def zip_dictionary(df, zip_col, city_col):
    df=ga_df.sort_values(by='city',ascending=False)[df[zip_col].notna()]
    zip_dict = dict(zip(df[city_col], df[zip_col]))
    return zip_dict

#Creating dictionary for each zip code
zip_dict1 = zip_dictionary(ga_df,'zip','city')
zip_dict2 = zip_dictionary(ga_df, 'servicing_lender_zip','servicing_lender_city')
zip_dict3 = zip_dictionary(ga_df, 'project_zip', 'project_city')

#Combining zip code dictionaries
#zip_dict = zip_dict1 | zip_dict2 | zip_dict3
zip_dict = {**zip_dict1, **zip_dict2, **zip_dict3}

del zip_dict1
del zip_dict2
del zip_dict3

#Create function that replaces zip code with key from zip code dictionary
def zip_clean(df, zip_col, city_col):
    start = time.process_time() #Start timer
    df[zip_col] = df[city_col]
    df[zip_col] = df[zip_col].replace(zip_dict)
    print(f"{zip_col} cleaned in {(time.process_time() - start):.2f} seconds.") # End timer
    return df

#Invoking function on all our zip codes
ga_df = zip_clean(ga_df,'zip', 'city')
ga_df = zip_clean(ga_df, 'servicing_lender_zip','servicing_lender_city')
ga_df = zip_clean(ga_df, 'project_zip', 'project_city')

del zip_dict

ga_df['zip'] = ga_df['zip'].astype(str)
ga_df

/var/folders/s9/wy2hq96d67x4vmq_tdh3c8tw0000gn/T/ipykernel_11358/1846180825.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df=ga_df.sort_values(by='city',ascending=False)[df[zip_col].notna()]


zip cleaned in 10.90 seconds.
servicing_lender_zip cleaned in 11.24 seconds.
project_zip cleaned in 10.86 seconds.


,loan_number,name,amount,state,address,city,zip,naics_code,business_type,jobs_retained,...,project_state,project_zip,originating_lender_city,originating_lender_state,loan_status_date,originating_lender_location_id,lmi_indicator,forgiveness_amount,forgiveness_date,Removed
0,2519198508,LUMMUS CORPORATION,2000000.0,GA,225 BOURNE BLVD,SAVANNAH,31419,333249.0,CORPORATION,294,...,GA,31419,CHICAGO,IL,NaT,116998,N,NaN,NaT,0
1,9918008300,COLIANT SOLUTIONS INC.,1294555.0,GA,2703 BRICKTON NORTH DR,BUFORD,30519,541519.0,CORPORATION,63,...,GA,30519,COLUMBUS,GA,NaT,19248,Y,NaN,NaT,0
2,9662228509,YOHE PLUMBING INC,729509.0,GA,1120 FRANKE COURT N/A,AUGUSTA,30909,238220.0,CORPORATION,105,...,GA,30909,PHOENIXVILLE,PA,NaT,121536,N,NaN,NaT,0
3,1661788406,LEWIS COLOR LITHOGRAPHERS INC,571193.4,GA,30 JOE KENNEDY BLVD,STATESBORO,30458,323111.0,CORPORATION,49,...,GA,30458,COLUMBUS,GA,NaT,19248,N,NaN,NaT,0
4,1217957101,"ALMA PAK INTERNATIONAL, LLC",472700.0,GA,230 PINEVIEW RD,ALMA,31510,445230.0,PARTNERSHIP,37,...,GA,31510,ALMA,GA,2021-01-08,17811,N,475877.59,2020-12-15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579659,4662698703,DONNELL WILSON,1950.0,GA,1000 PACES LN,WOODSTOCK,30188,621511.0,SOLE PROPRIETORSHIP,1,...,GA,30188,LAGUNA HILLS,CA,2021-04-01,509316,N,NaN,NaT,1
579660,5057699009,MAMADOU DIOULDE DIALLO,1849.0,GA,7000 ROSWELL RD APT 252,ATLANTA,30331,485310.0,INDEPENDENT CONTRACTORS,1,...,GA,30331,BEDFORD,TX,2021-05-21,529472,N,NaN,NaT,1
579661,8820408702,ANTHONY HARRIS,1245.0,GA,2849 BLOUNT ST,ATLANTA,30331,485310.0,INDEPENDENT CONTRACTORS,1,...,GA,30331,BEDFORD,TX,2021-04-08,529472,Y,NaN,NaT,1
579662,8028788903,TAAHIRA ROBINSON,1075.0,GA,8541 GLENDEVON CT,RIVERDALE,30296,561110.0,LIMITED LIABILITY COMPANY(LLC),1,...,GA,30296,MIAMI,FL,2021-05-11,529192,Y,NaN,NaT,1


### Adjusting Null Values

To clean some of the remaining null values, we used `.fillna`. For null values in NAICS code, we used "81" which according to the [NAICS website](https://www.naics.com/six-digit-naics/?code=81) indicates a business other than public service.

In [7]:
ga_df["name"].fillna("NAME UNAVAILABLE", inplace = True)
ga_df["address"].fillna("ADDRESS UNAVAILABLE", inplace = True)
ga_df["naics_code"].fillna(81, inplace = True)
ga_df["business_type"].fillna("OTHER", inplace = True)
ga_df["undisbursed_amount"].fillna(0, inplace = True) 
ga_df["forgiveness_amount"].fillna(0, inplace = True)


## Merging More Data

To make our analysis more insightful, we decided to include addtional data:
* [NAICS code names](https://data.world/sasha/2017-2022-sic-to-naics-crosswalk/workspace/file?filename=2017-2022+SIC+to+NAICS+crosswalk.csv)
* Median Income per Zip Code
* Population per Zip Code
* Number of people under the poverty line per Zip Code
* Race per Zip Code
* Latitude and Longitude for each Zip Code

*All out data, besides the NAICS code names, were retrived from the US Census Bureau and American community service on https://data.census.gov/*

### NAICS Code Names

In [8]:
path = os.path.expanduser("~/Documents/Code Projects/IC22_PPPLoan/Additional Data/")
naics = pd.read_excel(path + 'data_2017-2022 SIC to NAICS crosswalk.xlsx')

#Create function to rename columns
def new_name(df, old_name, new_name):
    df.rename(columns = {old_name:new_name}, inplace = True)
    return df

new_name(naics,'NAICS', 'naics_code')
new_name(naics,'NAICS Description', 'naics_name')

naics = naics[['naics_code', 'naics_name']]

#Add naics name to ga_df
naics_dict = dict(naics[['naics_code', 'naics_name']].values)
del naics

ga_df['naics_name'] = ga_df.naics_code.map(naics_dict)


### Demographics Data

In [9]:
# Loading supplemental files
ga_income = pd.read_csv(path + 'data_ACSST5Y2019.S1901_data_with_overlays_2022-03-01T222855_income.csv') #income per zip
ga_population = pd.read_csv(path + 'data_ACSDT5Y2019.B01003_data_with_overlays_2022-02-10T182445_population.csv') #population per zip
ga_poverty = pd.read_csv(path + 'data_ACSST5Y2019.S1701_data_with_overlays_2021-12-08T224622_poverty.csv') #poverty per zip
ga_race = pd.read_csv(path + 'data_ACSDT5Y2019.B02001_data_with_overlays_2022-02-04T110910_race.csv') #race per zip
us_zips = pd.read_excel(path + 'data_US_zip_codes_2013.xlsx') #latitude and longitude

#group removed df by zip codes
x = ga_df.groupby(by='zip').mean()

#Create function to clean US census dataframes
def clean_census(df):
    df.drop(index=df.index[0], axis=0, inplace=True) 
    df['NAME'] = df['NAME'].str[6:]
    df.rename(columns = {'NAME':'zip'}, inplace=True)
    return df
#Create function to convert columns into numbers
def numeric(df, col):
    df[col] = pd.to_numeric(df[col])
    return df

#Cleaning income df
clean_census(ga_income)
new_name(ga_income, 'S1901_C01_012E','median_income')
ga_income = ga_income[ga_income.median_income != '-']
numeric(ga_income, 'median_income') # household income
ga_income = ga_income[['zip', 'median_income']]

#Cleaning population df
clean_census(ga_population)
new_name(ga_population, 'B01003_001E','population')
numeric(ga_population, 'population')
ga_population = ga_population[['zip', 'population']]

#Cleaning poverty df
clean_census(ga_poverty)
new_name(ga_poverty, 'S1701_C02_001E','poverty_status')
numeric(ga_poverty, 'poverty_status')
ga_poverty = ga_poverty[['zip', 'poverty_status']]

#Cleaning race df
clean_census(ga_race)

new_name(ga_race,'B02001_002E','white')
new_name(ga_race,'B02001_003E','black')
new_name(ga_race,'B02001_004E','native')
new_name(ga_race,'B02001_005E','asian')
new_name(ga_race,'B02001_006E','hawaiian_pac_islander')
new_name(ga_race,'B02001_007E','other')
new_name(ga_race,'B02001_008E','mixed')

numeric(ga_race,'white')
numeric(ga_race,'black')
numeric(ga_race,'native')
numeric(ga_race,'asian')
numeric(ga_race,'hawaiian_pac_islander')
numeric(ga_race,'other')
numeric(ga_race,'mixed')

ga_race = ga_race[['zip', 'white', 'black', 'native', 'asian', 'hawaiian_pac_islander', 'other', 'mixed']]

ga_census = [ga_income, ga_population, ga_poverty, ga_race]

ga_census = reduce(lambda left,right: pd.merge(left,right,on='zip'), ga_census)

#merge grouped zip code df and census df
ga_census_full = pd.merge(x, ga_census,on="zip", how="left")

With the race data, we decided to calculate the percentage of minority and majority populations per zip code. While it is relatively discretionary, it's safe to assume white is the only race part of the majority population. 

In [10]:

ga_census_full = ga_census_full[
    ['zip', 
    'amount', 
    'jobs_retained', 
    'term', 
    'initial_approval_amount', 
    'current_approval_amount', 
    'undisbursed_amount', 
    'forgiveness_amount', 
    'Removed', 
    'median_income', 
    'population', 
    'poverty_status', 
    'white', 
    'black', 
    'native', 
    'asian', 
    'hawaiian_pac_islander', 
    'other', 
    'mixed']]

new_name(us_zips, 'ZIP','zip')
us_zips['zip'] = us_zips['zip'].astype(str)

ga_census_full = pd.merge(ga_census_full, us_zips,on="zip", how="left")
new_name(ga_census_full, 'LAT','lat')
new_name(ga_census_full, 'LNG','long')


#Additional columns
ga_census_full['majority_percent'] = ga_census_full["white"]/ga_census_full["population"]
ga_census_full["minority_percent"] = ga_census_full.iloc[:, 13:19].sum(axis=1)/ga_census_full["population"]
ga_census_full["poverty_percent"] = ga_census_full["poverty_status"]/ga_census_full["population"]
ga_census_full = ga_census_full.dropna()

del ga_income
del ga_population
del ga_poverty
del ga_race

#Adding to ga_df
ga_more = ga_census_full[['zip', 'lat', 'long','population', 'majority_percent', 'minority_percent','median_income' ,'poverty_percent']]
ga_df = pd.merge(ga_df, ga_more,on="zip", how="left")
ga_df.head()

,loan_number,name,amount,state,address,city,zip,naics_code,business_type,jobs_retained,...,forgiveness_date,Removed,naics_name,lat,long,population,majority_percent,minority_percent,median_income,poverty_percent
0,2519198508,LUMMUS CORPORATION,2000000.0,GA,225 BOURNE BLVD,SAVANNAH,31419,333249.0,CORPORATION,294,...,NaT,0,Other Industrial Machinery Manufacturing,31.997154,-81.230649,58800.0,0.514422,0.485578,57181.0,0.117568
1,9918008300,COLIANT SOLUTIONS INC.,1294555.0,GA,2703 BRICKTON NORTH DR,BUFORD,30519,541519.0,CORPORATION,63,...,NaT,0,Other Computer Related Services,34.087437,-83.944991,47685.0,0.675726,0.324274,88493.0,0.026801
2,9662228509,YOHE PLUMBING INC,729509.0,GA,1120 FRANKE COURT N/A,AUGUSTA,30909,238220.0,CORPORATION,105,...,NaT,0,"Plumbing, Heating, and Air-Conditioning Contra...",33.470481,-82.083045,41261.0,0.451758,0.548242,52247.0,0.143186
3,1661788406,LEWIS COLOR LITHOGRAPHERS INC,571193.4,GA,30 JOE KENNEDY BLVD,STATESBORO,30458,323111.0,CORPORATION,49,...,NaT,0,Commercial Printing (except Screen and Books),32.399676,-81.828266,44329.0,0.568364,0.431636,40011.0,0.300503
4,1217957101,"ALMA PAK INTERNATIONAL, LLC",472700.0,GA,230 PINEVIEW RD,ALMA,31510,445230.0,PARTNERSHIP,37,...,2020-12-15,0,Fruit and Vegetable Markets,31.545485,-82.440693,9730.0,0.787256,0.212744,37925.0,0.219733


In [11]:
ga_df['diversity_rate'] = ga_df['minority_percent'].round(decimals = 1)


In [12]:
ga_df

,loan_number,name,amount,state,address,city,zip,naics_code,business_type,jobs_retained,...,Removed,naics_name,lat,long,population,majority_percent,minority_percent,median_income,poverty_percent,diversity_rate
0,2519198508,LUMMUS CORPORATION,2000000.0,GA,225 BOURNE BLVD,SAVANNAH,31419,333249.0,CORPORATION,294,...,0,Other Industrial Machinery Manufacturing,31.997154,-81.230649,58800.0,0.514422,0.485578,57181.0,0.117568,0.5
1,9918008300,COLIANT SOLUTIONS INC.,1294555.0,GA,2703 BRICKTON NORTH DR,BUFORD,30519,541519.0,CORPORATION,63,...,0,Other Computer Related Services,34.087437,-83.944991,47685.0,0.675726,0.324274,88493.0,0.026801,0.3
2,9662228509,YOHE PLUMBING INC,729509.0,GA,1120 FRANKE COURT N/A,AUGUSTA,30909,238220.0,CORPORATION,105,...,0,"Plumbing, Heating, and Air-Conditioning Contra...",33.470481,-82.083045,41261.0,0.451758,0.548242,52247.0,0.143186,0.5
3,1661788406,LEWIS COLOR LITHOGRAPHERS INC,571193.4,GA,30 JOE KENNEDY BLVD,STATESBORO,30458,323111.0,CORPORATION,49,...,0,Commercial Printing (except Screen and Books),32.399676,-81.828266,44329.0,0.568364,0.431636,40011.0,0.300503,0.4
4,1217957101,"ALMA PAK INTERNATIONAL, LLC",472700.0,GA,230 PINEVIEW RD,ALMA,31510,445230.0,PARTNERSHIP,37,...,0,Fruit and Vegetable Markets,31.545485,-82.440693,9730.0,0.787256,0.212744,37925.0,0.219733,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579659,4662698703,DONNELL WILSON,1950.0,GA,1000 PACES LN,WOODSTOCK,30188,621511.0,SOLE PROPRIETORSHIP,1,...,1,Medical Laboratories,34.123528,-84.457649,64836.0,0.849065,0.150935,81282.0,0.068758,0.2
579660,5057699009,MAMADOU DIOULDE DIALLO,1849.0,GA,7000 ROSWELL RD APT 252,ATLANTA,30331,485310.0,INDEPENDENT CONTRACTORS,1,...,1,Taxi Service,33.707455,-84.544149,66658.0,0.015737,0.984263,48347.0,0.206922,1.0
579661,8820408702,ANTHONY HARRIS,1245.0,GA,2849 BLOUNT ST,ATLANTA,30331,485310.0,INDEPENDENT CONTRACTORS,1,...,1,Taxi Service,33.707455,-84.544149,66658.0,0.015737,0.984263,48347.0,0.206922,1.0
579662,8028788903,TAAHIRA ROBINSON,1075.0,GA,8541 GLENDEVON CT,RIVERDALE,30296,561110.0,LIMITED LIABILITY COMPANY(LLC),1,...,1,Office Administrative Services,33.563185,-84.441151,28169.0,0.076219,0.923781,57676.0,0.149526,0.9


## Returning Clean Full & Removed Datasets

Finally, after cleaning our data we split the dataset again by Removed and Full and exported to `.csv` files.

In [13]:
# Sorting by index
ga_df = ga_df.sort_index()

# Removing any duplicates based on index: loan number
ga_df = ga_df[~ga_df.index.duplicated(keep='first')]

# Updating dataframes 
ga_full_df = ga_df[ga_df["Removed"] == 0]
ga_removed_df = ga_df[ga_df["Removed"] == 1]


In [14]:
ga_full_df.to_csv('ppp-ga-full-clean.csv')
ga_removed_df.to_csv('ppp-ga-removed-clean.csv')
ga_df.to_csv('ppp-ga-clean.csv')